In [1]:
import random

import numpy as np
import pandas as pd

import torch

torch.cuda.empty_cache()

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

In [2]:
# Фиксируем random seed
SEED = 42
seed_all(SEED)

In [3]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = DATA_PATH / Path('cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

import pandas as pd

pd.set_option('display.max_colwidth', 500) 

In [4]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_path)

In [5]:
from app.models.translate_model import TranslationModel

In [6]:
model = TranslationModel(
    model_name="facebook/m2m100_418M",
    # model_name="facebook/m2m100_1.2B",
    cache_dir=DATA_CACHE,
    device="cpu"
)

In [7]:
from pathlib import Path
import json

long_text_path = Path("long_text.json")

with open(long_text_path, "r", encoding="utf-8") as f:
    data = json.load(f)

texts = data["texts"]
source_lang = data["source_lang"]
target_langs = data["target_langs"]
data

{'texts': ['Глава 1. Призыв к битве\n\nМир древности был полон чудес и чудовищ. В те дни на земле великих правителей и легендарных героев, сражения не ограничивались только оружием и армиями. Они были испытаниями силы духа, мужества и ума. Одним из таких испытаний стали гонки на колесницах, что стало невероятно популярным в различных уголках мира, и даже в Иудее, где собрались великие герои Библии.\n\nЭти гонки, однако, не были простыми. Они не были лишь забава ради. Каждая колесница олицетворяла не только военную мощь, но и внутренний бой, борьбу с судьбой, и даже божественные вмешательства. Колесницы, запряженные сильными животными, мчались по древним трассам, где каждое движение было решающим. Гонщики, ставшие участниками этих состязаний, были героями, чьи имена известны всем — Моисей, Давид, Илия и многие другие.\n\nГлава 2. Моисей и его колесница\n\nМоисей, великий вождь и пророк, был на старости лет, но в его глазах горел огонь непокорного духа. Он решил принять участие в этих го

In [8]:
text = texts[0]
target_lang = target_langs[0]

In [9]:
blocks = model.split_text_to_blocks(
    text=text,
    source_lang=source_lang,
    max_tokens=128,
    buffer=8
)
blocks

['Глава 1.',
 'Призыв к битве Мир древности был полон чудес и чудовищ.',
 'В те дни на земле великих правителей и легендарных героев, сражения не ограничивались только оружием и армиями.',
 'Они были испытаниями силы духа, мужества и ума.',
 'Одним из таких испытаний стали гонки на колесницах, что стало невероятно популярным в различных уголках мира, и даже в Иудее, где собрались великие герои Библии.',
 'Эти гонки, однако, не были простыми.',
 'Они не были лишь забава ради.',
 'Каждая колесница олицетворяла не только военную мощь, но и внутренний бой, борьбу с судьбой, и даже божественные вмешательства.',
 'Колесницы, запряженные сильными животными, мчались по древним трассам, где каждое движение было решающим.',
 'Гонщики, ставшие участниками этих состязаний, были героями, чьи имена известны всем — Моисей, Давид, Илия и многие другие.',
 'Глава 2.',
 'Моисей и его колесница Моисей, великий вождь и пророк, был на старости лет, но в его глазах горел огонь непокорного духа.',
 'Он решил

In [10]:
blocks[11]

'Моисей и его колесница Моисей, великий вождь и пророк, был на старости лет, но в его глазах горел огонь непокорного духа.'

In [11]:
model_inputs = model.tokenize(
    blocks[11], 
    source_lang, 
    max_length=128,
)
model_inputs['input_ids'].shape, model_inputs['attention_mask'].shape

(torch.Size([1, 45]), torch.Size([1, 45]))

In [12]:
decoded_text = model.tokenizer.decode(
    model_inputs["input_ids"][0],
    skip_special_tokens=False
)
print(decoded_text)

__ru__ Моисей и его колесница Моисей, великий вождь и пророк, был на старости лет, но в его глазах горел огонь непокорного духа.</s>


In [13]:
generated_tokens = model.model.generate(
    **model_inputs,
    forced_bos_token_id=model.tokenizer.lang_code_to_id['en'],
    max_length=256,
    num_beams=5,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    temperature=1.0,
    do_sample=False,
    early_stopping=True,
    # length_penalty=1.0,
    no_repeat_ngram_size=3,
)


d:\projects\env_ml\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [14]:
# Декодируем сгенерированные токены
generated_text = model.tokenizer.decode(
    generated_tokens[0],  # Берем первый элемент, если вывод многомерный
    skip_special_tokens=True  # Обычно лучше пропускать спwециальные токены
)

generated_text

'Moses and his wheelchair Moses, the great leader and prophet, were old years old, but in his eyes a fire of unacceptable spirit burned.'

In [15]:
translate = model._translate(
    text=blocks[11],
    source_lang=source_lang,
    target_lang='en',
    # repetition_penalty=1.0,
    # temperature=0.1,
    # input_max_length=128,
    # output_max_length=256,
    # num_beams=5,
    # early_stopping=False,
    # do_sample=True,
    # length_penalty=1.0,
)
translate

'Moses and his wardrobe Moses, the great leader and prophet, were old years old, but in his eyes a fire of the unrighteous spirit burned.'

In [16]:
result = model.translate(
    text=blocks[11],
    source_lang=source_lang,
    target_lang='en'
)
result

'Moses and his wardrobe Moses, the great leader and prophet, were old years old, but in his eyes a fire of the unrighteous spirit burned.'

In [17]:
# results = model.translate_batch(
#     texts=texts,
#     source_lang=source_lang,
#     target_langs=target_langs
# )

# for i, result in enumerate(results):
#     print(f"\n🔹 Текст {i+1}")
#     for lang, translation in result.items():
#         print(f"[{lang}] ({len(translation)} символов):\n{translation[:1000]}...")  # первые 1000 символов
